### Constant-NVE Simulation in Extended Phase-Space

- **System**: alanine dipeptide in vacuum
- **Force field**: AMBER ff03
- **Constraints**: Bonds to hydrogen atoms
- **Hydrogen mass repartitioning**: 3 Da
- **Phase-space extension**: Angular variables $\phi_s$ and $\psi_s$ attached to backbone dihedrals $\phi$ and $\psi$, respectively

This example aims to demonstrate the correctness of the implemented Extended Phase-Space (XPS) simulation approach by checking energy conservation.

In [ ]:
import io

import cvpack
import numpy as np
import openmm as mm
import openxps as xps
import pandas as pd

from matplotlib import pyplot as plt
from openmm import app, unit

In [ ]:
pdb = app.PDBFile("alanine-dipeptide.pdb")
physical_system = app.ForceField("amber03.xml").createSystem(
    pdb.topology,
    nonbondedCutoff=1.0 * unit.nanometer,
    switchDistance=0.8 * unit.nanometer,
    constraints=app.HBonds,
    hydrogenMass=3 * unit.dalton,
)
platform = mm.Platform.getPlatformByName("Reference")

In [ ]:
physical_time_step = 2 * unit.femtoseconds
dv_time_step = 16 * unit.femtoseconds
platform = mm.Platform.getPlatformByName("Reference")

The two Ramachandran angles $\phi({\bf r})$ and $\psi({\bf r})$ are taken as collective
variables (CVs) and associated with two new dynamical variables (DVs) $\phi_s$ and
$\psi_s$, respectively.

In [ ]:
backbone_atoms = [
    atom.index for atom in pdb.topology.atoms() if atom.name in ["C", "CA", "N"]
]
phi = cvpack.Torsion(*backbone_atoms[0:4], name="phi")
psi = cvpack.Torsion(*backbone_atoms[1:5], name="psi")

mass = 5 * unit.dalton * (unit.nanometer / unit.radians) ** 2
phi_s = xps.DynamicalVariable("phi_s", unit.radian, mass, xps.bounds.CIRCULAR)
psi_s = xps.DynamicalVariable("psi_s", unit.radian, mass, xps.bounds.CIRCULAR)

The coupling between the CVs and DVs is achieved by adding a harmonic potential
to the Hamiltonian.

In [ ]:
kappa = 1000 * unit.kilojoule_per_mole / unit.radian**2
coupling = xps.HarmonicCoupling(phi, phi_s, kappa) + xps.HarmonicCoupling(
    psi, psi_s, kappa
)

In [ ]:
xps_integrator = xps.SplitIntegrator(
    xps.integrators.VelocityVerletIntegrator(physical_time_step),
    xps.integrators.VelocityVerletIntegrator(dv_time_step),
)

To execute the XPS simulation, we create an `ExtendedSpaceSimulation` object with the dynamical variables and the coupling potential.

In [ ]:
simulation = xps.ExtendedSpaceSimulation(
    pdb.topology,
    xps.ExtendedSpaceSystem(physical_system, coupling),
    xps_integrator,
    platform,
)
simulation.context.setPositions(pdb.positions)
cv_values = coupling.getCollectiveVariableValues(simulation.context)
simulation.context.setDynamicalVariableValues([cv_values["phi"], cv_values["psi"]])

The total energy consists of the potential energy of the physical system (including the harmonic coupling potential) and the kinetic energy of both the physical system and extra dynamical variables.

An OpenMM reporter is used to compute and output these terms at every 10 steps. A StringIO object is used to capture the output for later analysis.

In [ ]:
stream = io.StringIO()
interval = 10

cv_reporter = cvpack.reporting.StateDataReporter(
    stream,
    interval,
    step=True,
    time=True,
    potentialEnergy=True,
    kineticEnergy=True,
    writers=[
        xps.ExtensionWriter(
            kinetic=True,
            dynamical_variables=True,
            forces=True,
            collective_variables=True,
            coupling_functions=True,
        ),
    ],
    speed=True,
)
simulation.reporters = [cv_reporter]

The total energy should be conserved along the simulation.

In [ ]:
total_time = 100 * unit.picoseconds

simulation.step(round(total_time / xps_integrator.getStepSize()))

In [ ]:
stream.seek(0)
data = pd.read_csv(stream)
data

Plotting the energy terms and the total energy to check the energy conservation.

In [ ]:
time = data["Time (ps)"]
potential = data["Potential Energy (kJ/mole)"]
physical_kinetic = data["Kinetic Energy (kJ/mole)"]
extra_kinetic = data["Extension Kinetic Energy (kJ/mole)"]
total = potential + physical_kinetic + extra_kinetic

fig, ax = plt.subplots()
ax.plot(time, potential, label="Potential Energy")
ax.plot(time, physical_kinetic, label="Physical Kinetic Energy")
ax.plot(time, extra_kinetic, label="Extra Kinetic Energy")
ax.plot(time, total, label="Total Energy")
ax.axhline(
    y=total[0], color="gray", linestyle="--", linewidth=1, label="Initial Total Energy"
)

fig.legend(loc="center left", bbox_to_anchor=(1, 0.5))
ax.set_xlabel("Time (ps)")
ax.set_ylabel("Energy (kJ/mol)")
ax.set_title("Constant NVE Simulation in an Extended Phase Space")
plt.show()

Plotting the trajectory of the DVs and their attached CVs to check the coupling between them.

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(8, 8))


def plot_trajectory(ax, data, angle):
    dv_traj = np.unwrap(data[f"{angle}_s (radian)"])
    delta = data[f"{angle} (radian)"] - dv_traj
    cv_traj = dv_traj + delta - 2 * np.pi * np.round(delta / (2 * np.pi))
    ax.plot(time, dv_traj, label=rf"$\{angle}_s$")
    ax.plot(time, cv_traj, label=rf"$\{angle}({{\bf r}})$")
    ax.set_xlabel("Time (ps)")
    ax.set_ylabel("Angle (radians)")
    ax.set_title(rf"Trajectory: $\{angle}({{\bf r}})$ and $\{angle}_s$")
    ax.legend(loc="upper right")


plot_trajectory(ax1, data, "phi")
plot_trajectory(ax2, data, "psi")

fig.suptitle("Constant NVE Simulation in an Extended Phase Space", y=1.03)
plt.tight_layout()
plt.show()